In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
from rt_analysis import rt_filter_map_plot
# import build_speedmaps_index

from IPython.display import display, Markdown, Latex, HTML
import json

In [ ]:
analysis_date = dt.date(2023, 4, 12)

In [ ]:
%%capture_parameters
human_date = analysis_date.strftime('%B %d %Y (%A)')
human_date

In [ ]:
from tqdm.notebook import tqdm

# Frequent SHN Routes

* quickly look for frequent transit routes using the State Highway Network
* also evaluate applicable district-sourced ideas

### CT Links:

https://app.smartsheet.com/folders/whC3J79cMJCf3MqQ7MGv8pxHgQ3JQ68VwGQGwX21

https://caltrans-my.sharepoint.com/:p:/g/personal/audrey_ogden_dot_ca_gov/EZbU16vBqYZHvrORsT-LzwsBBxJCZqfUPaKHmm5uqWmn_Q?e=4%3A4Ijf9v&CID=80D7A50B-5D1F-497F-8C92-9435B5782132&wdLOR=c3E29B53E-4F87-436E-BEDA-3F4281A53835


# SFMTA

In [ ]:
## parameters cell
itp_id = 282

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
rt_day.set_filter(route_names = ['43', '28', '29'])

In [ ]:
_m = rt_day.segment_speed_map()

## Muni Lombard

In [ ]:
rt_day.autocorridor(shape_id = '2856', stop_seq_range = [27, 32])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

## dev new frequency

In [ ]:
## filter 6a to 9p -- "all day" to include times of likely congestion
df = rt_day._filter(rt_day.stop_delay_view)

# min_time = 
(df
     >> filter(_.actual_time > dt.datetime.combine(rt_day.analysis_date, dt.time(6)),
               _.actual_time < dt.datetime.combine(rt_day.analysis_date, dt.time(21))
              )
     >> group_by(_.shape_id, _.route_id, _.route_short_name, _.trip_id)
     >> summarize(runtime = _.actual_time.max() - _.actual_time.min(),
                 start = _.actual_time.min())
     >> group_by(_.route_id, _.route_short_name)
     >> summarize(p50_runtime_minutes =_.runtime.quantile(.5).seconds / 60,
                  p20_runtime_minutes =_.runtime.quantile(.2).seconds / 60,
                  p80_runtime_minutes =_.runtime.quantile(.8).seconds / 60,
                 first_start = _.start.min(), last_start = _.start.max(),
                 n_trips = _.shape[0])
     >> mutate(span = _.last_start - _.first_start, span_minutes = _.span.map(lambda x: x.seconds) / 60,
              daily_avg_headway = (_.span_minutes / _.n_trips) * 2) # create avg per direction by multiplying 2x
).round(1)

In [ ]:
all_corridors = []

In [ ]:
all_corridors += [rt_day.corridor]

## Muni 19th A

In [ ]:
rt_day.autocorridor(shape_id = '2808', stop_seq_range = [21, 32])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)
## geoms error, no time to fix

## Muni 19th B

In [ ]:
rt_day.autocorridor(shape_id = '2808', stop_seq_range = [32, 37])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

## GGT

In [ ]:
## parameters cell
itp_id = 127

In [ ]:
muni_corr = rt_day.corridor.copy() # use same corridor from muni analysis

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
# rt_day.set_filter(route_names = ['43', '28', '29'])

In [ ]:
## actually fix this bug sometime?
_m = rt_day.segment_speed_map()

## GGT Lombard

In [ ]:
rt_day.add_corridor(muni_corr)

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.transit_priority_target_mph = 20 # one route exceeds even this on corridor!

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

In [ ]:
all_corridors += [rt_day.corridor]

# AC Transit

In [ ]:
## parameters cell
itp_id = 4

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
rt_day.set_filter(route_names = ['72', '72M', '72R'])

In [ ]:
_m = rt_day.segment_speed_map()

## AC Transit San Pablo

In [ ]:
rt_day.autocorridor(shape_id = 'shp-72R-15', stop_seq_range = [9, 19])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

In [ ]:
all_corridors += [rt_day.corridor]

# BBB

In [ ]:
## parameters cell
itp_id = 300

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
rt_day.set_filter(route_names = ['3', 'R3'])

In [ ]:
_m = rt_day.segment_speed_map()

In [ ]:
_m

## BBB Lincoln

In [ ]:
rt_day.autocorridor(shape_id = '26171', stop_seq_range = [16, 29])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

# LA Metro

In [ ]:
## parameters cell
itp_id = 182

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
rt_day.set_filter(route_names = ['4'])

In [ ]:
_m = rt_day.segment_speed_map()

## Metro Santa Monica Bl

In [ ]:
rt_day.autocorridor(shape_id = '40338_DEC22', stop_seq_range = [40, 50])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

In [ ]:
all_corridors += [rt_day.corridor]

# VTA

In [ ]:
## parameters cell
itp_id = 294

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
rt_day.set_filter(route_names = ['22', 'Rapid 522'])

In [ ]:
_m = rt_day.segment_speed_map()

In [ ]:
_m

## VTA ECR

In [ ]:
rt_day.autocorridor(shape_id = '107193', stop_seq_range = [44, 98])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

# Samtrans

In [ ]:
## parameters cell
itp_id = 290

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
rt_day.set_filter(route_names = ['ECR'])

In [ ]:
_m = rt_day.segment_speed_map()

## Samtrans ECR

In [ ]:
rt_day.autocorridor(shape_id = 'ECR1095', stop_seq_range = [3, 65])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
# m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

# m

## Part B

In [ ]:
rt_day.autocorridor(shape_id = 'ECR1098', stop_seq_range = [4, 35])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

# Foothill

In [ ]:
## parameters cell
itp_id = 112

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
rt_day.set_filter(route_names = ['188'])

In [ ]:
_m = rt_day.segment_speed_map()

## Foothill Foothill Bl

In [ ]:
rt_day.autocorridor(shape_id = '12286_shp', stop_seq_range = [1779, 2834])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

# From Project List
https://app.smartsheet.com/reports/4gRrr282hJ3MPVG7hcqX4XQHh2FR4GgpfW6pq5Q1?view=grid

In [ ]:
# shared_utils.rt_utils.show_full_df((pd.read_parquet('./_rt_progress_2023-05-17.parquet')
#                                    >> filter(_.status == 'map_confirmed')
#                                    )
#                                   )

In [ ]:
analysis_date = dt.date(2023, 5, 17)

# Eureka

In [ ]:
import importlib
importlib.reload(rt_filter_map_plot)

In [ ]:
## parameters cell
itp_id = 108

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
# rt_day.set_filter(route_names = ['188'])

In [ ]:
_m = rt_day.segment_speed_map()

## Eureka US101 (Broadway Truesdale - 4th per [RTP](https://www.hcaog.net/sites/default/files/vroom_2022-2042_full_report.pdf))

* difficult to analyze -- need to exclude distant stops from corridor (add to algorithm?)
* standardize on most common trip pattern

In [ ]:
rt_day.autocorridor(shape_id = 'p_178050', stop_seq_range = [13, 13.5],
manual_exclude={'p_179049': {'max': 4}, 'p_179044': {'max': 4}})
# intercity service, exclude stops distant from corridor

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

In [ ]:
all_corridors += [rt_day.corridor]

# TART SR267/89 -- no data

# Tahoe Transportation District US50

* limited data geographic extent

In [ ]:
## parameters cell
itp_id = 331

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
_m = rt_day.segment_speed_map()

In [ ]:
rt_day.autocorridor(shape_id = 'p_497358', stop_seq_range = [0, 20])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

In [ ]:
all_corridors += [rt_day.corridor]

# Fresno -- unable eval (new services)

# VCTC (SR126 Fillmore - Ventura)

* minimal thru running delay on available data...

In [ ]:
## parameters cell
itp_id = 380

In [ ]:
analysis_date = dt.date(2023, 4, 12)

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
_m = rt_day.segment_speed_map()

In [ ]:
_m

# LA Metro Projects

* NoHo-Pas BRT to only briefly use SR134, light delay
* As per sheet, NSFV improvements only intersect SHN
* Inglewood connector, Arbor Vitae not on SHN...
* Games Route Network, etc unable to analyze

In [ ]:
## parameters cell
itp_id = 331

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
_m = rt_day.segment_speed_map()

In [ ]:
rt_day.autocorridor(shape_id = 'p_497358', stop_seq_range = [0, 20])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

# Victor Valley

* service runs, difficult to evaluate
* 0.5/hour, limited span

# Others

* OCTA speculative, new services unable to eval

# OCTA SR39

* multiple SHN sections, eval longest
* about 5 trips/hour combined!

In [ ]:
## parameters cell
itp_id = 235

In [ ]:
%%capture
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
%%capture_parameters
organization_name = rt_day.organization_name
organization_name

In [ ]:
_m = rt_day.segment_speed_map()

In [ ]:
rt_day.autocorridor(shape_id = '2902', stop_seq_range = [5, 51])

In [ ]:
rt_day.corridor_stop_delays >> distinct(_.route_short_name)

In [ ]:
rt_day.corridor_metrics()

In [ ]:
rt_day.quick_map_corridor()

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds', no_title=False, shn=True, corridor=True)

In [ ]:
m

In [ ]:
all_corridors += [rt_day.corridor]